Code to build a basic soil water budget model based on the mathematical equations used by IDC with simplification as here the key is identifying percolation rather than applied water.

In [ ]:
# standard python utilities
import os
from os.path import join, exists, dirname, basename, expanduser
import glob
import sys
import time

import pandas as pd
import numpy as np

import h5py
from scipy.io import loadmat

# standard python plotting utilities
import matplotlib as mpl
import matplotlib.pyplot as plt

# standard geospatial python utilities
import shapely
import geopandas as gpd



In [ ]:
# import cvxpy as cp

In [ ]:
doc_dir = os.getcwd()
while basename(doc_dir) != 'Documents':
    doc_dir = dirname(doc_dir)
# dir of all gwfm data
gwfm_dir = dirname(doc_dir)+'/Box/research_cosumnes/GWFlowModel'
gwfm_dir
dis_dir = join(gwfm_dir, 'DIS_data')
uzf_dir = join(gwfm_dir,'UZF_data')

In [ ]:
proj_dir = join(dirname(doc_dir),'Box','SESYNC_paper1')
data_dir = join(proj_dir, 'model_inputs')

In [ ]:
# run_dir = 'C://WRDAPP/GWFlowModel'
run_dir = 'F://WRDAPP/GWFlowModel'
loadpth = run_dir +'/Cosumnes/levee_setback/streamflow/'
# model_nam = 'setback_streamflow'
model_nam = 'historical_streamflow'

loadpth = run_dir +'/Cosumnes/Regional/'

model_ws = loadpth + 'crop_modflow'

In [ ]:
year = int(2014)
# crop='Grape'
crop='Corn'
# crop='Alfalfa'
# crop='Pasture' # will require extra work due to AUM vs hay
# crop = 'Misc Grain and Hay'

In [ ]:
# dem_data = np.loadtxt(gwfm_dir+'/DIS_data/dem_52_9_200m_mean.tsv')


In [ ]:
# nrow, ncol = dem_data.shape
nrow,ncol = (100, 230)

In [ ]:
def ymd2dt(year, month, day, year_adj):
    """ Only year is expected to come as int the others as series/1D arrays"""
    year = (year + year_adj).astype(str)
    date = pd.to_datetime(year+'-'+ month.astype(str)+'-'+day.astype(str))
    return(date)


## Hydrologic data

In [ ]:
fn = join(data_dir,'static_model_inputs.xlsx')
var_gen = pd.read_excel(fn, sheet_name='General', comment='#')
var_gen = var_gen.set_index('variable')['value'] # adjust for quick pulling of variables

var_crops = pd.read_excel(fn, sheet_name='Crops', comment='#')
var_yield = pd.read_excel(fn, sheet_name='Yield', comment='#')
# subset to crop for current simulation if looping, could also do ID
var_crops = var_crops[var_crops.crop==crop]
var_crops = var_crops.set_index('variable')['value'] # adjust for quick pulling of variables
var_yield = var_yield[var_yield.crop==crop]


# cross-reference between simple and DWR full crop name
crop_dict = pd.read_excel(fn, sheet_name='Name_dict', comment='#')
crop_dict = crop_dict.set_index('Crop')['DWR_name'].to_dict()

# season dates - primarily for calculating yield adjustments
season = pd.read_excel(fn, sheet_name='Seasons', comment='#')
season = season[season.crop==crop]
#  get the dates for each yield cycle
yield_start = ymd2dt(year, season.month_start, season.day_start, season.start_adj)
yield_end = ymd2dt(year, season.month_end, season.day_end, season.end_adj)
# adjust for dates in the next year
# yield_end[yield_start>yield_end] += pd.DateOffset(years=1)
# get the total extent of the irrigation season (calculation period)
strt_date = yield_start.min()
end_date = yield_end.max()

In [ ]:
# days to index for calculating yield impacts, add 1 to include end day
yield_ind = np.append([0], (yield_end-strt_date).dt.days.values +1)
yield_ind


In [ ]:
# the model will run the irrigation optimizer on specified dates (multiple crops can be done at once or in sequence)
# the modflow model will be run for the periods between the specified irrigation optimizer dates

# strt_date = ymd2dt(year, season.month_start, season.day_start)
# end_date = ymd2dt(year, season.month_end, season.day_end)
dates = pd.date_range(strt_date, end_date, freq='D')

model_ws = loadpth + 'crop_soilbudget/'+crop+'_'+str(strt_date.date())

In [ ]:
nper = (end_date-strt_date).days +1


In [ ]:
## Potential ETo spatial interpolation from CIMIS
fn = glob.glob(join(uzf_dir,'CIMIS','Cosumnes_dailyET_precip*.csv'))
daily_data = pd.DataFrame()
for file in fn:
    new_data = pd.read_csv(file, index_col = ['Date'], parse_dates = True)
    daily_data = pd.concat((daily_data, new_data))
# units of mm
data_in = daily_data[daily_data['Stn Name']=='Fair Oaks']

# clean up data so columns are by location, units of Precip are in mm
rain_in = data_in.pivot_table(index = 'Date', columns = 'Stn Name', values = 'Precip (mm)')
rain_m = rain_in/1000
# fill in missing days of rain with zero values
rain_df = rain_m.reindex(dates)
rain_df[rain_df.isna()]=0
# convert to array for calculations
rain = rain_df['Fair Oaks'].values

# clean up ETo data
ETo_in = data_in.pivot_table(index = 'Date', columns = 'Stn Name', values = 'ETo (mm)')
ETo_m = ETo_in/1000
# linearly interpolate missing days for ET
ETo_df = ETo_m[strt_date:end_date].resample('D').interpolate('linear')['Fair Oaks']

# create array for every period of rainfall
# rain_df = rain_m[strt_date:end_date].resample('D').interpolate('linear')['Fair Oaks']
# rain_arr = np.repeat(np.repeat(np.reshape(rain_df.values, (rain_df.shape[0],1,1)), nrow, axis=1),ncol, axis=2)


In [ ]:
fn = join(data_dir,'static_model_inputs.xlsx')
Kc = pd.read_excel(fn, sheet_name='Kc', comment='#')
Kc = Kc.set_index('Period')
Kc_dates = pd.read_excel(fn, sheet_name='Kc_dates', comment='#')
Kc_dates = Kc_dates[Kc_dates.Growth_stage!='Growing season'] # don't need overall dates for Kc calculation
Kc_dates['Start_date'] = pd.to_datetime({'year': year, 'month':Kc_dates['Start_month'], 'day': Kc_dates['Start_day']})
Kc_dates['End_date'] = pd.to_datetime({'year': year, 'month':Kc_dates['End_month'], 'day': Kc_dates['End_day']})
Kc_dates.Start_date = pd.to_datetime(Kc_dates.Start_date)
Kc_dates.End_date = pd.to_datetime(Kc_dates.End_date)
Kc_dates = Kc_dates.set_index('Growth_stage')


In [ ]:
Kc_c = Kc[Kc.Crop==crop]
Kc_dates_c = Kc_dates[Kc_dates.Crop==crop]

In [ ]:
def get_Kc_dates(Kc_dates_c, Kc_c):
    """ Given a dataframe with the dates for Kc periods and a dataframe with those Kc values create a timeseries
    of Kc values"""
    Kc_df_all = pd.DataFrame()
    # initial period and mid season are constant, crop dev and late season are linear
    # initial period
    dates_p = pd.date_range(Kc_dates_c.loc['Initial period', 'Start_date'], Kc_dates_c.loc['Initial period', 'End_date'])
    Kc_df = pd.DataFrame(dates_p, columns=['date']).assign(Kc=Kc_c.loc['ini', 'Value'])
    Kc_df_all = pd.concat((Kc_df_all, Kc_df))
    # crop development
    dates_p = pd.date_range(Kc_dates_c.loc['Crop development', 'Start_date'], Kc_dates_c.loc['Crop development', 'End_date'])
    Kc_df = pd.DataFrame(dates_p, columns=['date'])
    Kc_df['Kc'] = np.linspace(Kc_c.loc['ini', 'Value'], Kc_c.loc['mid', 'Value'], len(Kc_df))
    Kc_df_all = pd.concat((Kc_df_all, Kc_df))
    # mid-season
    dates_p = pd.date_range(Kc_dates_c.loc['Mid-season', 'Start_date'], Kc_dates_c.loc['Mid-season', 'End_date'])
    Kc_df = pd.DataFrame(dates_p, columns=['date']).assign(Kc=Kc_c.loc['mid', 'Value'])
    Kc_df_all = pd.concat((Kc_df_all, Kc_df))
    # late season
    dates_p = pd.date_range(Kc_dates_c.loc['Late season', 'Start_date'], Kc_dates_c.loc['Late season', 'End_date'])
    Kc_df = pd.DataFrame(dates_p, columns=['date'])
    Kc_df['Kc'] = np.linspace(Kc_c.loc['mid', 'Value'], Kc_c.loc['end', 'Value'], len(Kc_df))
    Kc_df_all = pd.concat((Kc_df_all, Kc_df))
    return(Kc_df_all)

In [ ]:
Kc_df = get_Kc_dates(Kc_dates_c, Kc_c)
# get the crop ET
ETc = ETo_df.values*Kc_df.Kc.values

In [ ]:
# load in pre-processed array of ETc for all time
ETc_long = pd.read_hdf(join(uzf_dir, "dwr_ETc",'long_ETc_all_lu.hdf5'), key='variable')
# identify crop to use
etc_var = pd.Series(ETc_long.variable.unique())
# etc_var[etc_var.str.contains(crop, case=False)]
# crop_dict = {'Corn':'Corn (field & sweet)'}

In [ ]:
# etc_var[etc_var.str.contains('Grain', case=False)]
# # ETc_long[['Grain and hay crops']]
# ETc_long[ETc_long.variable=='Grain and hay crops']['value'][strt_date:end_date].plot()

In [ ]:
# ETc_long.variable[ETc_long.variable.str.contains('vineyards',case=False)]

While the database of Kc will work fine for the rest of the crops it may make sense to use Yusuke's specific Kc values.


In [ ]:
# # subset for crop
# ETc = ETc_long[ETc_long.variable==crop_dict[crop]]['value']
# # subset for model period
# ETc = ETc[strt_date:end_date]
# # fill in empty dates with interpolation (temporary since corn dates are slightly different)
# ETc = ETc.reindex(dates)
# ETc = ETc.bfill().ffill()

# # convert to an array for calculations
# ETc = ETc.values

In [ ]:
in_2_m = (1/12)*0.3048

In [ ]:
d_ini = 0; # Initial root zone depletion (in)
y_max = var_crops[['y_max']].values # Maximum expected crop yield (tons/acre)

phi = var_gen['phi'] # Energy requirement to raise a unit of water by a unit of vertical distance (kWh/acre-in/ft)
p_c = var_crops['p_c'] # Crop price ($/ton)
p_sw = var_gen['p_sw'] # Surface water charges and fees ($/acre-in)
p_e = var_gen['p_e'] # Cost of energy for groundwater pumping ($/kWh)
p_o = var_crops['p_o'] # Variable operating costs per acre, excluding irrigation costs ($/acre)



In [ ]:
# create time series of daily yield response factors
var_yield['dt'] = ymd2dt(year, var_yield.month, var_yield.day, var_yield.year_adj)
# adjust for dates in the next year
K_Y = var_yield.set_index('dt').reindex(dates).ffill()
K_Y = K_Y['K_Y'].values


In [ ]:

gap_irr = var_crops['gap_irr'] # Number of days between irrigations
n_irr = np.floor(len(dates)/gap_irr).astype(int) + 1 # Calculate number of irrigations
irr_days = np.arange(0, (n_irr*gap_irr-1), gap_irr).astype(int) # Calculate days on which irrigation takes place
irr_days

# Spatial data

In [ ]:
# load cleaned soil data for ag fields
soil_path = join(uzf_dir,'clean_soil_data')
# soil data for each ag field
soil_ag_all = pd.read_csv(join(soil_path, 'soil_for_ag_fields.csv'), index_col=0)
# connection of ag fields to grid cells
grid_soil = pd.read_csv(join(soil_path, 'ag_field_to_cell.csv'),index_col=0)
# curve numbers
CN = pd.read_csv(join(soil_path, 'ag_field_CN.csv'),index_col = 0)

soil_ag_all = pd.merge(soil_ag_all, CN)
field_area = soil_ag.field_area_m2.values

# identify crop to use
crop_nam = pd.Series(soil_ag_all.name.unique())
crop_nam[crop_nam.str.contains(crop)]

# subset for fields with the crop
soil_crop = soil_ag_all[soil_ag_all.name==crop_dict[crop]]
nfield = len(soil_crop)

In [ ]:
## need to update to use land use by Yusuke
# ag_lu = gpd.read_file(join(uzf_dir,'county_landuse', 'domain_ag_lu_2018.shp'))
# # 5 duplicates in irrigation efficiency
# ag_irr_eff = ag_lu[['geom_id', 'name', 'irr_name', 'Avg_eff']].drop_duplicates()
# # # # irrigation efficiency
# soil_ag = soil_ag.merge(ag_irr_eff, how='left')
# # create irrigation efficiency multiplier to have 0 applied water for non-irrgated
# soil_ag['irr_eff_mult'] = 100/soil_ag.Avg_eff
# soil_ag.loc[soil_ag.irr_eff_mult.isna(), 'irr_eff_mult'] = 0


In [ ]:
# soil_ag_all[soil_ag_all.name.str.contains('grain and hay')].name.unique()

In [ ]:
# temporary input for dtw variables to calculate pumping cost
dtw_time = np.linspace(90,95, nper)
dtw_all = np.zeros((nper,nfield))
for n in np.arange(nper):
    dtw_all[n,:] = dtw_time[n]

## Iterate over each unique soil condition

In [ ]:
ns=0
soil_ag = soil_crop.iloc[[ns]] #keep as dataframe for consistency 
nfield = soil_ag.shape[0]

dtw_arr = dtw_all[:,ns]

etc_arr = np.zeros((nper,nfield))
for n in np.arange(nper):
    etc_arr[n,:] = ETc[n]

In [ ]:
def prep_soil(soil_ag, etc_arr):
    global soil_Ks, soil_por, soil_eps, soil_CN
    global soildepth, soil_m, wc_f, wc_wp, taw, Smax, raw, P
    
    # # when soil_K_low is missing using a substitute of Ksat/10
    soil_Ks = np.where(soil_ag.Ksat_Low==0, soil_ag.Ksat/10, soil_ag.Ksat_Low)
    soil_por = soil_ag.Porosity.values/100
    soil_eps = soil_ag.EPS.values
    soil_CN = soil_ag.CN.values
    
    soildepth = soil_ag.SoilDepth.values
    psdi =  soil_ag.PSDI.values
    # parameter for Mualem, van Genuchten
    soil_m = psdi/(psdi+1)
    wc_f =  soil_ag.w3rdbar.values/100 #field capacity
    wc_wp =  soil_ag.w15bar.values/100 #wilting point 

    # Calculate total available water in the root zone
    taw = (wc_f - wc_wp)*soildepth 

    # for runoff, convert CN from fraction to CN
    Smax = (1000/soil_CN) - 10
    
    p_table22 = var_crops['p_table22'] # Soil water depletion fraction for no stress
    # requires ETc in mm/day
    P = p_table22 + 0.04*((5-(etc_arr*1000))) # Calculate adjusted daily soil water depletion fraction for no stress
    raw = taw*P # Calculate readily available water in the root zone


prep_soil(soil_ag, etc_arr)

In [ ]:
def calc_S(wc, Smax, wc_f, soil_por):
    """ Given an array of water contents return potential soil retention"""
    S = np.zeros(Smax.shape)
    # when water content is less than 1/2 field capacity, full retention
    S[wc < wc_f/2] = Smax[wc < wc_f/2]
    # wc > porosity then set as porosity for calculations (avoids S >1)
    wc_calc = np.where(wc<soil_por, wc, soil_por)
    # when water content is greater than 1/2 field capacity, partial retention 
    S1 = Smax * (1 - ((wc_calc - wc_f/2)/(soil_por - wc_f/2)))
    S[wc >= wc_f/2]= S1[wc >= wc_f/2]
    # convert S from inches to meters
    S *= (0.3048/12)
    return(S)

In [ ]:
def calc_pc(wc, soil_por, soil_Ks, soil_m):
    """ explicit calculation of percolation assuming water content of prior to 
    percolation is representative of conditions controlling percolation"""
    # calculate soil saturation as water content can be greater than porosity assuming it represents ponding
    sat = wc/soil_por
    sat  = np.where(sat>1, 1, sat)
    # explicit calculation of percolation
    pc = soil_Ks*(sat)*(1- (1-(sat)**(1/soil_m))**soil_m)**2
    return(pc)

# calc_pc(wc[100], soil_por, soil_Ks, soil_m)

Total available water (TAW) is the water available between field content and the wilting point times the depth of the root zone (this could be assumed to be the soil depth at times).  
Per the FAO report, rooting depths are:
- Corn:1-1.7 m which is about where soil depth would put it  
- Alfalfa:1-2 m  
- Barley, Wheat: 1-1.5m  

So all of these fall within the range where the soil depth should determine rooting depth.

If we need to remove ET from each step then we should reduce the number of crops used to simplify calculations unless the code runs quickly for all cells.
- Need to assume initial water content of 0. 
 
* water content can't exceed porosity. It is assumed that θ can be computed to be greater than θT, and the difference between the θ and θT represents the depth of the pond.

# Run the model
The ETc, and rain will be preset inputs for the irrigation simulation (the ETc will be calculated when the crop is chosen at the start of the year).  
The irrigation optimizer will be a function of crop - ETc, irrigation (variable calibrated)

In [ ]:
irr_lvl = np.zeros(2*n_irr); # Initial irrigation values for optimization
irr_lvl[:] = (2/12)*0.3048 # irrigate with 2 inches (convert to meters)



After the irrigation season ends, the model needs to be run from the end of irrigation to the start of the next season for calculating recharge from rainfall and maintaining the soil water budget.  

The FAO model assumes "that water can be stored in the root zone until field capacity is reached. Although following heavy rain or irrigation the water content might temporally exceed field capacity, the total amount of water above field capacity is assumed to be lost the same day by deep percolation, following any ET for that day. By assuming that the root zone is at field capacity following heavy rain or irrigation, the minimum value for the depletion Dr, i is zero."  

-> this contrasts with IDC where water content is allowed to exceed porosity with the assumption that it has become ponded water.

In [ ]:
p_gw = p_e*phi*dtw_arr[irr_days]
# where the gw costs more then set gw as 0 and vice versa
irr_lvl[:n_irr][p_gw > p_sw] = 0
irr_lvl[n_irr:][p_gw < p_sw] = 0

In [ ]:

    
def swb(irr_lvl):
    global wc, pc, rp, ETa, D, K_S
#     global c_gwtot, c_swtot
    m2_ac = (1/0.3048**2)/43560 # convert from m2 to acres
    in_2_m = (1/12)*0.3048 # convert from inches to meters
    nper = (end_date-strt_date).days +1

    tic = time.time()

    irr_sw = np.zeros((nper,nfield))
    irr_gw = np.zeros((nper,nfield))
    for i in np.arange(0,n_irr):
        irr_sw[irr_days[i]] = irr_lvl[i]
        irr_gw[irr_days[i]] = irr_lvl[i+n_irr]
        
    wc = np.zeros((nper+1, nfield)) # water content, add initial conditions with +1
    pc = np.zeros((nper, nfield)) # percolation
    rp = np.zeros((nper, nfield)) # runoff 
    ETa = np.zeros((nper, nfield)) # actual ET
    wb_sum= np.zeros((nper, nfield)) # water budget check
    # time units are days for everything

    D = np.zeros((nper+1, nfield)) # soil depletion, add initial conditions with +1
    K_S = np.zeros((nper, nfield)) # crop water stress
    
    # initial water content and root zone depletion are pulled from the last step of the previous run
    
    # -1 starts at IC for BC
    # WC/D starts at 0
    for ns, n in enumerate(np.arange(-1, nper-1)):
        ## Runoff ##
        S = calc_S(wc[ns+1], Smax, wc_f, soil_por)
        water_in = rain[n+1] 
        # calculate runoff only when there is rain, and rain is uniform
        if (water_in>0).any():
            rp[n+1] = ((water_in - 0.2*S)**2)/(water_in + 0.8*S)
        # where rainfall is less than initial abstraction (0.2S) set runoff as 0
        rp[n+1] = np.where(water_in<0.2*S, 0, rp[n+1])
        # add in irrigation after runoff (assume farm is set up to avoid runoff for irrigation season)
        # *irr_eff_mult
        water_in = water_in + irr_sw[n+1] + irr_gw[n+1]
        ## explicit percolation ##
        pc[n+1] = calc_pc(wc[ns], soil_por, soil_Ks, soil_m)
        # stepwise water budget, explicit to avoid iteration
        # add rain and take away runoff first
        wc[ns+1] = (wc[ns]*soildepth + (water_in - rp[n+1]))/soildepth
        # take away ET, add term to prevent going to zero
        ETa[n+1] = np.where(ETc[n+1] <= wc[ns+1]*soildepth, ETc[n+1], wc[ns+1]*soildepth - 1E-9)
        wc[ns+1] = wc[ns+1] + (-ETa[n+1])/soildepth
        # take away percolation
        pc[n+1] = np.where(pc[n+1] <= wc[ns+1]*soildepth, pc[n+1], wc[ns+1]*soildepth - 1E-9)
        wc[ns+1] = wc[ns+1] + (-pc[n+1])/soildepth
        # check water budget error
        wb_sum[n+1] = (wc[ns]-wc[ns+1])*soildepth + water_in - rp[n+1] - ETa[n+1] - pc[n+1] 
        if (wb_sum[n+1]>1E-3).any()|(wb_sum[n+1]<-1E-3).any():
            print('WB error exceeds 1E-3',n )
            ## additional code for optimizing irrigation
        # calculate soil depletion for irrigation decision (must use ETc to see how much should be depleted)
        D[ns+1] = D[ns] - water_in + ETc[n+1] + rp[n+1] + pc[n+1] 
        # root zone depletion can't be greater than TAW 
        D[ns+1] = np.min([D[ns+1], taw], axis=0)
        # root zone depletion should be greater than 0
        D[ns+1] = np.where(D[ns+1]<0,0, D[ns+1])
        # default value of water stress is 1 (none): # potentially unnecessary just fill in 1
        K_S[n+1] = 1
        # where rootzone depletion is greater than RAW there is water stress
        K_S_ws = (taw - D[ns+1])/((1 - P[n+1])*taw);
        K_S[n+1] = np.where(D[ns+1]>raw[n+1], K_S_ws, K_S[n+1])

    ## Calculate daily crop outcomes 
    ETc_adj = np.transpose(K_S)*ETc # Calculate daily crop ET with soil water stress, pairwise functoin check?
    ## Calculate economic outcomes 
    arr1 = np.ones((nfield,nper))
    # average the yield scaling across the season
    # yield max changes during the season so update this to be fluid for alfalfa (mean for eaching cutting then sum means)
    Y_A_arr = np.zeros(len(y_max))
    for n in np.arange(0,len(yield_ind)-1):
        # subset the yield scaling by the growing period then multiply by the appropriate yield max for that period
        Y_A_arr[n] = y_max[n]*np.mean((arr1- (arr1 - (ETc_adj/ETc))*K_Y)[:, yield_ind[n]:yield_ind[n+1]])
    # the total yield is the sum of individual yields
    Y_A = Y_A_arr.sum()
    # irrigation must be converted from meters to inches for calculating cost
    c_gwtot = p_e*phi*(np.multiply(dtw_arr, irr_gw[:,0])/in_2_m) # Calcualte total groundwater pumping costs for the season ($/acre)
    c_swtot = np.multiply(p_sw, irr_sw[:,0])/in_2_m # Calcualte total surface water costs for the season ($/acre)
    cost = c_gwtot+c_swtot
    # calculate profit (daily values must be summed for the seasonal value)
    # profit can be left as $/acre with area assigned after if each field is done separate
    # for alfalfa it would be helpful to see the profit by cutting
    pi = -((p_c*Y_A - np.sum(cost))- p_o) # Calculate profit ($/acre)
    # forced internal boundary to prevent negatives
    if any(irr_lvl <0):
        # set a scalable penalty
        pi = irr_lvl[irr_lvl<0].sum()*-1E4
    
    toc = time.time()
#     print('Run time was %.2f minutes' %((toc-tic)/60))
    if wb_sum.sum(axis=1).mean() > 1E-6:
        print('Avg WB error was %.2E m' % wb_sum.sum(axis=(1)).mean())
    return(pi)


If Ks is never less than 1 then there is never water stress (D < RAW). Water stress does appear when lower irrigation levels are used.

I should double check transpose is doing what I expect with the first two cells

We had discussed that we could adjust the irrigation volumes to assign only SW or GW irrigation if the price of GW or SW is cheaper. The issue is that the price of GW will fluctuate throughout the season so at the beginning it might be cheaper but get more expensive by the end so it could reasonably put GW at the beginning and SW at the end. To do this we need to specify which irrigation events are more or less cost.

In [ ]:
for ns in [0]:#np.arange(0,len(soil_ag_unique)):
    soil_ag = soil_crop.iloc[[ns]] #keep as dataframe for consistency 
    nfield = soil_ag.shape[0]
    
    etc_arr = np.zeros((nper,nfield))
    for n in np.arange(nper):
        etc_arr[n,:] = ETc[n]

    prep_soil(soil_ag, etc_arr)
    dtw_arr = dtw_all[:,ns]
    irr_lvl = np.zeros(2*n_irr); # Initial irrigation values for optimization
    irr_lvl[:] = (2/12)*0.3048 # irrigate with 2 inches (convert to meters)
    
    p_gw = p_e*phi*dtw_arr[irr_days]
    # where the gw costs more then set gw as 0 and vice versa
    irr_lvl[:n_irr][p_gw > p_sw] = 0
    irr_lvl[n_irr:][p_gw < p_sw] = 0
    
    pi = swb(irr_lvl)
    print('Profit %.2f ($/acre)' %(-pi))

In [ ]:
irr_lvl


In [ ]:
def calc_yield(ETc, K_S, y_max, yield_ind,  nfield, nper):
    ## Calculate daily crop outcomes 
    ETc_adj = np.transpose(K_S)*ETc # Calculate daily crop ET with soil water stress, pairwise functoin check?
    ## Calculate economic outcomes 
    arr1 = np.ones((nfield,nper))
    # average the yield scaling across the season
    # yield max changes during the season so update this to be fluid for alfalfa (mean for eaching cutting then sum means)
    Y_A_arr = np.zeros(len(y_max))
    for n in np.arange(0,len(yield_ind)-1):
        # subset the yield scaling by the growing period then multiply by the appropriate yield max for that period
        Y_A_arr[n] = y_max[n]*np.mean((arr1- (arr1 - (ETc_adj/ETc))*K_Y)[:, yield_ind[n]:yield_ind[n+1]])
    # the total yield is the sum of individual yields
    Y_A = Y_A_arr.sum()
    return(Y_A)
    
calc_yield(ETc, K_S, y_max, yield_ind,  nfield, nper)

In [ ]:
def calc_profit(Y_A, p_c, p_e, phi, dtw_arr, irr_gw, p_sw, irr_sw):
    in_2_m = (1/12)*0.3048 # convert from inches to meters
    c_gwtot = p_e*phi*(np.multiply(dtw_arr, irr_gw[:,0])/in_2_m) # Calcualte total groundwater pumping costs for the season ($/acre)
    c_swtot = np.multiply(p_sw, irr_sw[:,0])/in_2_m # Calcualte total surface water costs for the season ($/acre)
    cost = c_gwtot+c_swtot
    # calculate profit (daily values must be summed for the seasonal value)
    pi = -((p_c*Y_A - np.sum(cost))- p_o) # Calculate profit ($/acre)
    # forced internal boundary to prevent negatives
    if any(irr_lvl <0):
        # set a scalable penalty, assuming p_o would be a sizable penalty
        pi = irr_lvl[irr_lvl<0].sum()*-p_o*10
    return(pi)
calc_profit(Y_A, p_c, p_e, phi, dtw_arr, irr_gw, p_sw, irr_sw)  

In [ ]:
## error checking
c_gwtot = p_e*phi*(np.multiply(dtw_arr, irr_gw[:,0])/in_2_m) # Calcualte total groundwater pumping costs for the season ($/acre)
c_swtot = np.multiply(p_sw, irr_sw[:,0])/in_2_m # Calcualte total surface water costs for the season ($/acre)
cost = c_gwtot+c_swtot
print('GW Cost %.2f' %c_gwtot.sum(),'SW Cost %.2f' %c_swtot.sum())

# # the yield scale is negative sometimes
# ETc_adj = np.transpose(K_S)*ETc
# arr1 = np.ones((nfield,nper))
# # average the yield scaling across the season
# Y_A_arr = np.zeros(len(y_max))
# for n in np.arange(0,len(yield_ind)-1):
#     # subset the yield scaling by the growing period then multiply by the appropriate yield max for that period
#     Y_A_arr[n] = y_max[n]*np.mean((arr1- (arr1 - (ETc_adj/ETc))*K_Y)[:, yield_ind[n]:yield_ind[n+1]])
# # the total yield is the sum of individual yields
# Y_A = Y_A_arr.sum()
Y_A = calc_yield(ETc, K_S, y_max, yield_ind,  nfield, nper)
p_c*Y_A - p_o - np.sum(cost)
p_c*Y_A, np.sum(cost), p_o


profit for alfalfa (1,000) seems a little excessive but operating costs are low and if 7 cuttings are done then it's a lot
vineyard profit is even greater (2,000)
corn only nets around 200 

Misc. grain and hay didn't find a positive solution (started -397 went -307) -> issue is the Kc values used were for summer crop not winter so there was no ETc to make yield

In [ ]:
# test = np.arange(0, 4, 0.5)
# out = np.zeros(len(test))
# for n,i in enumerate(test):
#     irr_lvl[:] = i
#     out[n] = swb(irr_lvl)

In [ ]:
from scipy.optimize import Bounds, LinearConstraint
from scipy.optimize import minimize

In [ ]:
ACON

In [ ]:
# Total surface water and groundwater available during the season (in)
# irr_tot = np.array([10, 10]) 
# to represent unconstrained conditions set boundaries at 100, 100
irr_tot = np.array([100, 100]) 
irr_tot = (irr_tot/12)*0.3048 # convert to meters
# Coefficients for inequality constraints (first n_irr columns are for surface water; second n_irr columns are for groundwater)
ACON = np.zeros((2,2*n_irr))
ACON[0,:n_irr] = np.ones(n_irr)
ACON[1,(n_irr):(2*n_irr)] = np.ones(n_irr)
# AEQCON = [] #  No equality contraints
# BEQCON = [] # No equality contraints
# I_WMAXLBCON = np.zeros((1,2*n_irr)) # Irrigation cannot be negative

# COBYLA can't use bounds so need extra inequality constraints
# pos_con = np.zeros((len(irr_lvl), len(irr_lvl)))
# np.fill_diagonal(pos_con,1)
# ACON = np.append(ACON, pos_con, axis=0)
# irr_tot = np.append(irr_tot, np.full(len(pos_con), 10))

con_min = np.zeros(len(ACON)) 


bounds = Bounds(lb = 0)
linear_constraint = LinearConstraint(ACON, list(con_min), list(irr_tot))

Notes on solvers from:
- fmincon from MATLAB was suggested anecodotally as faster than scipy.minimize
- someone suggested CVXPY as an alternate which is what Jon Herman had us use for non-linear convex problems

In [ ]:
# # for SLSQP
# ineq_cons = {'type':'ineq',
#             'fun': lambda x: np.array([irr_tot[0] - np.sum(x[:n_irr]),
#                                 irr_tot[1] - np.sum(x[n_irr:])]),
#              'jac': lambda x: np.array([[10],
#                                       [10]])
#             }

In [ ]:
# the minimization with 'trust-constr' and no constraints doesn't solve and has increasing WB error
# out = minimize(swb, irr_lvl, method='SLSQP',
# #         constraints = [ineq_cons],
#         bounds=bounds,
# #          options={'verbose':1}
#         )

In [ ]:
# 'trust-constr', 
#'SLSQP' and 'COBYLA' -> don't seem to work easily
out = minimize(swb, irr_lvl, method='trust-constr',
        constraints = [linear_constraint],
        bounds=bounds,
        tol = 0.0001
#          options={'verbose':1}
        )

Nelder-Mead isn't faster, fails to solve with 300 maxiter (can be bounded)  
CG - no bounds available, tried to set internal boundary but model accepted the penalty or after adjusting internal constraint it failed to solve  
-> forcing an internal constraint only serves to create more non-linearity rather than setting the solve space  

SLSQP was able to solve the problem with an internal 0-bound but it found a solution with primarily GW and a little bit of SW which doesn't make sense since GW is cheaper. This was not improved with finer tolerance (1E-2 times more).  
COBYLA was able to evaluate successfully after increasing maxiter to 600. Same issue as SLSQP where a mix of GW and SW are used despite no constraints.  

In [ ]:
# out = minimize(swb, irr_lvl, method='COBYLA',
# #         constraints = [linear_constraint],
# #         bounds=bounds,
#          options={'disp':1, 'maxiter':600},
#                tol=0.0001
#         )

The tolerance should be to about 1 mm (1 in = 25.4 mm) which requires tol = 0.001
Testing different tolerances:
- 0.1 evaluated to -250.9826
- 0.01 to -251.8
- 0.001 to -253.1
- 0.0001 to -250.26
- 0.00001 to -253.50609
- 0.000001 to -253.50609  

So far 0.1, 0.01 equally used up the water to the 3rd decimal. Also the testing was done using initial values near the final solution. 

In [ ]:
out.x[:n_irr].sum(), out.x[n_irr:].sum(), out.fun

In [ ]:
out.execution_time/60

In [ ]:
gw_irr_in= (out.x[:n_irr]/0.3048).sum()*12
sw_irr_in = (out.x[n_irr:]/0.3048).sum()*12
print('Irr (in) GW %.2f'%gw_irr_in, 'SW %.2f'%sw_irr_in)
print('Cost ($/acre) GW %.2f' %(c_gwtot.sum()),'SW %.2f'%(c_swtot.sum()))

In [ ]:
plt.bar(np.arange(0,n_irr)-.25/2,(out.x[:n_irr]/0.3048)*12, label='SW', width=0.25)
plt.bar(np.arange(0,n_irr)+.25/2,(out.x[n_irr:]/0.3048)*12, label='GW', width=0.25)
plt.legend()
plt.ylabel('Irrigation (in)')
plt.xlabel('Irrigation Event')
# out

In [ ]:
# cvxpy won't work with the same input, it seems that the function would need to be rewritten with cvxpy
# functions instead of numpy

# n = len(irr_lvl)
# x = cp.Variable(n)
# objective = cp.Minimize(swb(x))
# # constraints = [0 <= x]

# prob = cp.Problem(objective, constraints)

# print("Optimal value", prob.solve())
# print("Optimal var")
# print(x.value) # A numpy ndarray.

In [ ]:
# with constraints the model took 117 iterations
# the x values all seem to be miniscule 1E-6 to 1E-7
# function was -357647159
# took 470 seconds
# 1 inch = 0.0254 meters

In [ ]:
# 0.4 min @ 320 fields is 128 min just for corn which is excessive

# code works in the loop ( no negatives like above)

# for some reason the most recent run had run times in the minutes range for each optimization
# so the total runtime was 14 hours
# really need to figure out what the problem is with solving time

In [ ]:
# dtw_all.shape

In [ ]:
# save irrigation, function, time
irr_all =np.empty((len(soil_crop),len(irr_lvl)))
p_all = np.empty(len(soil_crop))
t_all = np.empty(len(soil_crop))

for ns in np.arange(0,len(soil_crop)):
    soil_ag = soil_crop.iloc[[ns]] #keep as dataframe for consistency 
    nfield = soil_ag.shape[0]

    dtw_arr = dtw_all[:,ns]

    etc_arr = np.zeros((nper,nfield))
    for n in np.arange(nper):
        etc_arr[n,:] = ETc[n]

    prep_soil(soil_ag, etc_arr)

    irr_lvl = np.zeros(2*n_irr); # Initial irrigation values for optimization
    irr_lvl[:] = (2/12)*0.3048 # irrigate with 2 inches (convert to meters)
    if ns > 1:
        irr_lvl[:] = irr_all[ns-1]
    # the minimization with 'trust-constr' and no constraints doesn't solve and has increasing WB error
    out = minimize(swb, irr_lvl, method='trust-constr',
            constraints = [linear_constraint],
            bounds=bounds,
    #          options={'verbose':1}
                   tol = 0.01
            )
    irr_all[ns] = out.x
    p_all[ns] = out.fun
    t_all[ns] = out.execution_time
    print('%.2f' %(-out.fun),'$ ,in %.2f' %(out.execution_time/60),'min')

In [ ]:
t_all.sum()/60/60

In [ ]:
lines =plt.plot(np.transpose(irr_all))
 

In [ ]:
fig,ax = plt.subplots(5,1,sharex=True)
ax[0].plot(rain)
# ax[0].plot(irr_sw[:,0])
ax[0].bar(x=np.arange(0,len(irr_sw))-0.25, height=irr_sw[:,0], width=1,label='SW')
ax[0].bar(x=np.arange(0,len(irr_gw))+0.25, height=irr_gw[:,0], width=.5, label='GW')
ax[1].plot(rp.mean(axis=(1)))
ax[2].plot(ETa.mean(axis=(1)))
ax[3].plot(pc.mean(axis=(1)))
ax[4].plot(wc.mean(axis=(1)))

for n, l in enumerate(['Rain/\nIrrigation','Runoff','ET','Perc','WC']):
    ax[n].set_ylabel(l)
ax[0].legend()

## Save Output

In [ ]:

import h5py

def arr_to_h5(arr, h5_fn):
    # convert arrays of annual rates to hdf5 files individually
    with h5py.File(h5_fn, "w") as f:
        grp = f.require_group('array') # makes sure group exists
        grp.attrs['units'] = 'meters/day'
        grp.attrs['description'] = 'Each layer of the array is a day in the water year'
        dset = grp.require_dataset('WY', arr.shape, dtype='f', compression="gzip", compression_opts=4)
        dset[:] = arr
    


In [ ]:
# years and array index 
years = pd.date_range(strt_date,end_date,freq='AS-Oct')
yr_ind = (years-strt_date).days


In [ ]:

# need separte hdf5 for each year because total is 300MB
for n in np.arange(0,len(yr_ind)-1):
    arr = pc[yr_ind[n]:yr_ind[n+1]]
    fn = join(model_ws,"percolation_WY"+str(years[n].year+1)+".hdf5")
    arr_to_h5(arr, fn)
